# Google Agent Development Kit (ADK) CookBook

### **Оглавление**

1. В чём философия ADK: модульность и контроль
2. Что такое `Agent` и как строить иерархии?
3. Что такое `Runner` и как он управляет выполнением?
4. Что такое `Tools` и как расширить возможности агента?
5. Что такое `Services` (Memory, Artifacts) и как управлять состоянием?
6. Что такое `Evaluation` и как тестировать агентов?

### **Введение в Google Agent Development Kit (ADK)**

**Google ADK** — это не просто ещё один фреймворк для создания агентов. Это целая философия, которая ставит во главу угла **модульность, контроль и строгие инженерные практики**. Если OpenAI Agents SDK предлагает элегантную простоту и быструю интеграцию, то ADK даёт тебе в руки конструктор LEGO, где каждый кубик можно заменить, настроить или расширить. Этот подход превращает разработку агентов из "чёрного ящика" в предсказуемый процесс, похожий на традиционную разработку ПО.

### ⚙️ Ключевые компоненты: что под капотом?

Архитектура ADK построена на принципах инверсии контроля и внедрения зависимостей. Вместо монолитного движка у нас есть ядро (`Agent`, `Runner`) и набор подключаемых сервисов.

```
┌───────────────────────────┐
│          Runner           │  <- Уровень исполнения и сессий
│ (Управляет жизненным циклом)│
└─────────────┬─────────────┘
              │
┌─────────────▼─────────────┐
│           Agent           │  <- Уровень логики (инструкции, модель)
│ (Может иметь sub-agents)  │
└─────────────┬─────────────┘
              │
┌─────────────▼─────────────┐
│          Services         │  <- Уровни поддержки (внедряемые зависимости)
│ (Memory, Tools, Artifacts)│
└───────────────────────────┘
```

### 💡 Почему это важно?
- **Гибкость**: Ты можешь заменить хранилище памяти с `InMemory` на кастомное решение для `Redis`, не меняя логику агента.
- **Контроль**: Ты точно знаешь, как чанкуются документы или как хранится история, потому что ты сам выбрал или написал этот сервис.
- **Тестируемость**: Каждый компонент можно тестировать изолированно, что критически важно для сложных систем.

### 🔮 Что дальше в туториале?
Мы с тобой разберём каждый из этих "кубиков" и посмотрим, как они соединяются вместе для создания надёжных и масштабируемых ИИ-агентов.

In [ ]:
# Основные импорты из Google ADK (гипотетические для примеров)
from google.adk.agents import LlmAgent, BaseAgent
from google.adk.runners import Runner
from google.adk.tools import tool
from google.adk.memory import BaseMemoryService, InMemoryMemoryService
from google.adk.evaluation import AgentEvaluator, TestCase

# Импорты для моделей данных и типизации
from pydantic import BaseModel, Field
from typing import Optional, List, Dict
import random

---

## 🤖 2. Что такое `Agent` и как строить иерархии?

`Agent` в ADK — это основной строительный блок, который инкапсулирует логику, инструкции и возможности конкретного исполнителя. Но главная его сила — в поддержке **иерархии**, что позволяет создавать сложные мульти-агентные системы.

### 🧠 Как устроен `Agent` под капотом

Когда ты создаёшь агента, ты, по сути, конфигурируешь его поведение. Ключевые классы:

*   `BaseAgent`: Абстрактный класс, определяющий базовый интерфейс (имя, описание).
*   `LlmAgent`: Конкретная реализация для агентов на базе LLM. Именно здесь ты указываешь модель, инструкции и инструменты.

Ключевое отличие ADK — атрибуты для построения иерархии:

| Атрибут | Назначение |
|---|---|
| `parent_agent` | Ссылка на родительского агента. |
| `sub_agents` | Список дочерних агентов, которым можно делегировать задачи. |

Эта структура позволяет агенту-диспетчеру (`triage_agent`) находить и передавать управление специализированным агентам (`developer_agent`, `qa_agent`).

### ✅ Как это работает

1.  **Определение**: Ты создаёшь экземпляры агентов, как обычные Python-объекты.
2.  **Связывание**: Ты явно указываешь связи, присваивая список дочерних агентов атрибуту `sub_agents` родителя.
3.  **Делегирование**: В промпте родительского агента ты даёшь инструкцию делегировать задачи подходящим `sub_agents` на основе их `description`. LLM генерирует внутреннюю команду на передачу управления, которую обрабатывает `Runner`.

In [ ]:
# =========================
# КОНТЕКСТ И МОДЕЛИ ДАННЫХ
# =========================
class ProjectManagementContext(BaseModel):
    """Контекст для агентов по управлению проектами."""
    current_ticket_id: Optional[str] = None
    user_role: str = "Viewer"
    open_tickets: List[str] = Field(default_factory=list)

# =========================
# ОПРЕДЕЛЕНИЕ АГЕНТОВ
# =========================

# Агент-разработчик, который может комментировать тикеты
developer_agent = LlmAgent(
    name="DeveloperAgent",
    description="Этот агент может добавлять технические комментарии к тикетам.",
    instructions="Ты — ИИ-ассистент разработчика. Добавляй комментарии к тикетам.",
    # tools=[add_comment_tool] # Инструмент будет определен позже
)

# Агент QA, который может изменять статус тикетов
qa_agent = LlmAgent(
    name="QAAgent",
    description="Этот агент может изменять статус тикета (например, 'Done', 'In Progress').",
    instructions="Ты — ИИ-ассистент QA. Изменяй статус тикетов.",
    # tools=[change_status_tool] # Инструмент будет определен позже
)

# Главный агент-диспетчер (триаж)
triage_agent = LlmAgent[ProjectManagementContext](
    name="TriageAgent",
    instructions=(
        "Ты — главный агент-диспетчер. Твоя задача — проанализировать запрос пользователя "
        "и делегировать его одному из твоих под-агентов на основе их описания. "
        "Если пользователь хочет добавить комментарий, используй DeveloperAgent. "
        "Если пользователь хочет изменить статус, используй QAAgent.",
    ),
    sub_agents=[developer_agent, qa_agent] # Явно строим иерархию
)

print(f"Главный агент: {triage_agent.name}")
print(f"Дочерние агенты: {[agent.name for agent in triage_agent.sub_agents]}")

---

## 🚀 3. Что такое `Runner` и как он управляет выполнением?

`Runner` — это **движок**, который приводит всю систему в движение. Он берёт на себя всю грязную работу по управлению жизненным циклом запроса: создаёт сессии, вызывает LLM, координирует вызовы инструментов и передачу управления между агентами.

### 🧠 Как устроен `Runner` под капотом

Когда ты вызываешь `Runner.run_sync(agent, input)`, происходит примерно следующее:

1.  **Создание сессии**: `Runner` инициализирует сессию выполнения, используя предоставленные сервисы (например, `InMemoryMemoryService`).
2.  **Цикл выполнения (Execution Loop)**:
    a. **Подготовка промпта**: `Runner` вызывает `agent.get_system_prompt()` и формирует полный промпт, включая историю из `MemoryService`.
    b. **Вызов LLM**: Отправляет запрос к модели, указанной в агенте.
    c. **Анализ ответа**: Ответ от LLM может быть:
        - **Финальным ответом**: Цикл завершается, результат возвращается пользователю.
        - **Вызовом инструмента (`tool_call`)**: `Runner` находит нужный инструмент в списке `agent.tools`, выполняет его и добавляет результат в историю. Цикл продолжается с шага (a).
        - **Командой на делегирование**: `Runner` находит нужного `sub_agent` и рекурсивно запускает для него цикл выполнения, передавая текущий контекст.
3.  **Завершение сессии**: Результаты сохраняются, сессия закрывается.

Ключевая идея в том, что `Agent` — это декларативная конфигурация, а `Runner` — это императивный исполнитель этой конфигурации.

In [ ]:
# =========================
# ЗАПУСК АГЕНТА
# =========================

# Для демонстрации мы пока не будем использовать реальные инструменты,
# а просто посмотрим на гипотетический вывод агента.

def mock_run(agent, user_input):
    """Мок-функция для имитации ответа LLM."""
    if "комментарий" in user_input.lower():
        return f"Делегирую задачу агенту: {developer_agent.name}"
    elif "статус" in user_input.lower():
        return f"Делегирую задачу агенту: {qa_agent.name}"
    return "Не могу определить подходящего агента.",

user_query_1 = "Добавь комментарий к тикету #123: 'Исправлено в новой версии'."
user_query_2 = "Измени статус тикета #123 на 'Done'."

# В реальном коде это был бы вызов Runner.run_sync(triage_agent, user_query_1)
print(f"Запрос: '{user_query_1}'")
print(f"Ответ triage_agent: {mock_run(triage_agent, user_query_1)}
")

print(f"Запрос: '{user_query_2}'")
print(f"Ответ triage_agent: {mock_run(triage_agent, user_query_2)}")

---

## 🛠️ 4. Что такое `Tools` и как расширить возможности агента?

Инструменты (`Tools`) — это то, что позволяет агентам взаимодействовать с внешним миром: вызывать API, работать с базами данных, выполнять вычисления. В ADK, как и в других фреймворках, это реализуется через предоставление LLM описания функций, которые она может попросить выполнить.

### 🧠 Как устроен `@tool` под капотом

Декоратор `@tool` — это синтаксический сахар, который превращает обычную Python-функцию в объект, понятный для `Runner` и LLM.

**Ты пишешь:**
```python
@tool
def get_ticket_status(ticket_id: str) -> str:
    '''Возвращает статус тикета по его ID.'''
    ...
```

**ADK делает следующее:**

1.  **Анализ функции**: Извлекает имя (`get_ticket_status`), описание (из докстринга) и параметры (`ticket_id: str`) с их типами.
2.  **Создание JSON-схемы**: Генерирует схему, которую можно передать в LLM. Это позволяет модели понять, какие аргументы ожидает функция.
    ```json
    {
      "name": "get_ticket_status",
      "description": "Возвращает статус тикета по его ID.",
      "parameters": {
        "type": "object",
        "properties": {
          "ticket_id": { "type": "string" }
        },
        "required": ["ticket_id"]
      }
    }
    ```
3.  **Создание объекта `Tool`**: Создаётся экземпляр класса `Tool`, который хранит эту схему и ссылку на исходную Python-функцию. Именно этот объект ты и передаёшь в список `tools` при создании агента.

In [ ]:
# =========================
# ОПРЕДЕЛЕНИЕ ИНСТРУМЕНТОВ
# =========================

# Гипотетическая база данных тикетов
TICKETS_DB = {
    "ADK-101": {"status": "Open", "comments": []},
    "ADK-102": {"status": "In Progress", "comments": ["Initial analysis done."]},
}

@tool
def add_comment_to_ticket(ticket_id: str, comment: str) -> str:
    """Добавляет комментарий к указанному тикету."""
    if ticket_id in TICKETS_DB:
        TICKETS_DB[ticket_id]["comments"].append(comment)
        return f"Комментарий успешно добавлен к тикету {ticket_id}."
    return f"Ошибка: Тикет {ticket_id} не найден."

@tool
def change_ticket_status(ticket_id: str, new_status: str) -> str:
    """Изменяет статус указанного тикета."""
    if ticket_id in TICKETS_DB:
        TICKETS_DB[ticket_id]["status"] = new_status
        return f"Статус тикета {ticket_id} изменен на '{new_status}'."
    return f"Ошибка: Тикет {ticket_id} не найден."

# =========================
# ОБНОВЛЕНИЕ АГЕНТОВ С ИНСТРУМЕНТАМИ
# =========================

developer_agent.tools = [add_comment_to_ticket]
qa_agent.tools = [change_ticket_status]

print(f"Инструменты агента {developer_agent.name}: {[t.name for t in developer_agent.tools]}")
print(f"Инструменты агента {qa_agent.name}: {[t.name for t in qa_agent.tools]}")

---

## 📦 5. Что такое `Services` (Memory, Artifacts) и как управлять состоянием?

Сервисы — это сердце модульной архитектуры ADK. Это сменные компоненты, которые отвечают за управление состоянием, хранение данных и другие сквозные задачи. В отличие от OpenAI, где управление состоянием (Threads) — это "магия" внутри API, в ADK ты явно контролируешь этот процесс.

### 🧠 Как устроены `Services` под капотом

Каждый сервис — это реализация абстрактного базового класса.

1.  **`BaseMemoryService`**: Определяет интерфейс для управления памятью диалога (`add_to_history`, `get_history`).
    *   **`InMemoryMemoryService`**: Простая реализация, которая хранит историю в памяти. Идеальна для тестов и простых приложений.
    *   **Кастомная реализация**: Ты можешь написать `RedisMemoryService`, который будет хранить историю в Redis, обеспечивая персистентность между перезапусками.

2.  **`BaseArtifactService`**: Определяет интерфейс для работы с артефактами — файлами, которые агент может создавать или читать (`save`, `load`, `list`).
    *   **`InMemoryArtifactService`**: Хранит файлы в памяти.
    *   **`GcsArtifactService`**: Реализация для хранения артефактов в Google Cloud Storage. 

При запуске `Runner` ты можешь передать ему экземпляры нужных сервисов. Если не передать, используются реализации по умолчанию (обычно `InMemory`).

In [ ]:
# =========================
# РАБОТА С СЕРВИСАМИ
# =========================

# 1. Создаем экземпляр сервиса памяти
memory_service = InMemoryMemoryService()

# 2. Добавляем что-то в историю для определенной сессии
session_id = "session_123"
memory_service.add_to_history(session_id, {"role": "user", "content": "Привет!"})
memory_service.add_to_history(session_id, {"role": "agent", "content": "Здравствуйте! Чем могу помочь?"})

# 3. Получаем историю
history = memory_service.get_history(session_id)

print(f"История для сессии {session_id}:")
for message in history:
    print(f"  - {message['role']}: {message['content']}")

# В реальном приложении экземпляр сервиса передается в Runner,
# который автоматически управляет историей в течение сессии.
# runner = Runner(memory_service=memory_service, ...)


---

## 🧪 6. Что такое `Evaluation` и как тестировать агентов?

Это одна из самых сильных и уникальных сторон ADK. Фреймворк предоставляет встроенные инструменты для **систематического тестирования и оценки** агентов. Это позволяет применять к разработке агентов проверенные инженерные практики, такие как TDD (Test-Driven Development).

### 🧠 Как устроена `Evaluation` под капотом

1.  **`TestCase`**: Класс для описания одного тестового случая. Обычно включает:
    *   `input`: Входные данные для агента.
    *   `expected_output`: Ожидаемый результат или критерии его оценки.

2.  **`AgentEvaluator`**: Основной класс-оценщик. Его метод `evaluate()` принимает:
    *   `agent`: Агент, которого нужно протестировать.
    *   `dataset`: Список объектов `TestCase`.

3.  **Процесс оценки**: `AgentEvaluator` итерируется по набору данных, для каждого `TestCase` запускает `Runner` с соответствующим `input`, а затем сравнивает фактический результат с `expected_output`.

4.  **Результат**: По итогам оценки генерируется отчет, показывающий, какие тесты прошли успешно, а какие — нет, с детальной информацией об ошибках.

Этот подход позволяет автоматически проверять, не сломалась ли логика агента после внесения изменений, и объективно измерять его качество.

In [ ]:
# =========================
# ОЦЕНКА АГЕНТОВ
# =========================

# 1. Создаем тестовые случаи
eval_dataset = [
    TestCase(
        input="Добавь коммент к ADK-101: 'срочно исправить'",
        expected_output="Комментарий успешно добавлен к тикету ADK-101.",
    ),
    TestCase(
        input="Какой статус у тикета ADK-102?",
        # Здесь мы можем ожидать не точный текст, а наличие подстроки
        expected_output="In Progress",
    ),
    TestCase(
        input="Поменяй статус ADK-102 на 'Done'",
        expected_output="Статус тикета ADK-102 изменен на 'Done'.",
    )
]

# 2. Создаем оценщика
# В реальном коде мы бы передали сюда функцию для сравнения результатов
evaluator = AgentEvaluator()

print("Запускаем оценку агента... (симуляция)")

# 3. Запускаем оценку (симуляция)
# В реальности был бы вызов: results = evaluator.evaluate(triage_agent, eval_dataset)

def mock_evaluate(agent, dataset):
    results = []
    # Гипотетически, здесь должен быть сложный вызов triage_agent, который делегирует
    # задачу, а мы проверяем конечный результат. Для простоты сделаем прямые вызовы.
    res1 = add_comment_to_ticket("ADK-101", "срочно исправить")
    res3 = change_ticket_status("ADK-102", "Done")
    
    results.append({"input": dataset[0].input, "output": res1, "expected": dataset[0].expected_output, "passed": res1 == dataset[0].expected_output})
    results.append({"input": dataset[1].input, "output": "In Progress", "expected": dataset[1].expected_output, "passed": "In Progress" == dataset[1].expected_output})
    results.append({"input": dataset[2].input, "output": res3, "expected": dataset[2].expected_output, "passed": res3 == dataset[2].expected_output})
    return results

evaluation_results = mock_evaluate(triage_agent, eval_dataset)

for i, result in enumerate(evaluation_results):
    status = '✅ PASSED' if result['passed'] else '❌ FAILED'
    print(f"--- Тест #{i+1} ---")
    print(f"Вход: {result['input']}")
    print(f"Выход: {result['output']}")
    print(f"Ожидалось: {result['expected']}")
    print(f"Статус: {status}
")